In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load the historical closing prices from CSV file
data = pd.read_csv('/kaggle/input/stock-data/Nifty_50_with_indicators_.csv')
data['Date']=pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')
data = data.set_index(pd.DatetimeIndex(data['Date'].values))
data=data[['Date', 'open', 'high','low', 'close']]
data

In [ ]:
# Calculate RSI
def calculate_rsi(data, window=14):
    close_prices = data['close'].values
    changes = np.diff(close_prices)
    up_changes = np.where(changes >= 0, changes, 0)
    down_changes = np.where(changes < 0, -changes, 0)
    avg_gain = np.mean(up_changes[:window])
    avg_loss = np.mean(down_changes[:window])
    rsi_values = []
    for i in range(window, len(close_prices)+window-2):
        if avg_loss == 0:
            rsi = 100
        else:
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))
        rsi_values.append(rsi)
        change = changes[i - window + 1]
        if change >= 0:
            avg_gain = (avg_gain * (window - 1) + change) / window
            avg_loss = (avg_loss * (window - 1)) / window
        else:
            avg_gain = (avg_gain * (window - 1)) / window
            avg_loss = (avg_loss * (window - 1) - change) / window
        
    print(len(close_prices))
    return np.array(rsi_values)

In [ ]:
# Prepare features and target
closing_prices = data['close'].values
rsi_values = calculate_rsi(data)
print(closing_prices)
print(rsi_values)
X = rsi_values.reshape(-1, 1)
x=[]
y = closing_prices[:-2]

In [ ]:
# split data into train test sets
splitlimit = int(len(X)*0.8)
print(splitlimit)
X_train, X_test = X[:splitlimit], X[splitlimit:]
y_train, y_test = y[:splitlimit], y[splitlimit:]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train)

In [ ]:
# Train the linear regression model
model = LinearRegression()
model.fit(np.array(X_train), y_train)

# Make predictions
predictions = model.predict(X_test)

# Print predicted and actual closing prices
for i in range(len(predictions)):
    print("Predicted:", predictions[i])
    print("Actual:", y_test[i])

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(y_test, color = 'black', label = 'Test')
plt.plot(predictions, color = 'green', label = 'pred')
plt.legend()
plt.show()